In [3]:
import pandas as pd
import zipfile
import requests
from io import BytesIO
import os

In [6]:
print("Columns in the DataFrame:")
print(df.columns)

Columns in the DataFrame:
Index(['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')


In [7]:
df.shape[0]

3799407

In [8]:
df.dtypes


No disposition                  int64
Date mutation                  object
Nature mutation                object
Valeur fonciere                object
No voie                       float64
B/T/Q                          object
Type de voie                   object
Code voie                      object
Voie                           object
Code postal                   float64
Commune                        object
Code departement               object
Code commune                    int64
Prefixe de section            float64
Section                        object
No plan                         int64
No Volume                      object
1er lot                        object
Surface Carrez du 1er lot      object
2eme lot                       object
Surface Carrez du 2eme lot     object
3eme lot                       object
Surface Carrez du 3eme lot     object
4eme lot                      float64
Surface Carrez du 4eme lot     object
5eme lot                       object
Surface Carr

In [9]:
# change float to int for code postal
df = df.dropna(subset=['Code postal'])
df['Code postal'] = df['Code postal'].astype(str)

C:\Users\yutin\AppData\Local\Temp\ipykernel_13548\3363316655.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Code postal'] = df['Code postal'].astype(str)


# List of postal codes for Montpellier Méditerranée Métropole
codes_postaux_metropole = [
    "34670", "34130", "34160", "34170", "34830", "34660",
    "34690", "34790", "34990", "34970", "34920", "34980",
    "34000", "34070", "34080", "34090", "34570", "34470",
    "34730", "34680", "34430", "30440", "34740"
]


In [10]:
# List of postal codes for Montpellier Méditerranée Métropole
codes_postaux_metropole = [
    "34670", "34130", "34160", "34170", "34830", "34660",
    "34690", "34790", "34990", "34970", "34920", "34980",
    "34000", "34070", "34080", "34090", "34570", "34470",
    "34730", "34680", "34430", "30440", "34740"
]

# isin() pour filtrage 
code_mask = df['Code postal'].isin(codes_postaux_metropole)
df_metropoleMP =df[code_mask]

In [11]:
df_metropoleMP.head()

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,...,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain


In [12]:
df_metropoleMP.shape[0]


0

In [14]:
# List of postal codes for Montpellier Méditerranée Métropole
codes_postaux_metropole = [
    "34670", "34130", "34160", "34170", "34830", "34660",
    "34690", "34790", "34990", "34970", "34920", "34980",
    "34000", "34070", "34080", "34090", "34570", "34470",
    "34730", "34680", "34430", "30440", "34740"
]

In [51]:
#les données suivantes sont les données dans herault  (34)
url0 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2024/departements/34.csv.gz" # 2024 (seulement Q1)
url1 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2023/departements/34.csv.gz" # 2023
url2 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2022/departements/34.csv.gz" # 2022
url3 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2021/departements/34.csv.gz" # 2021
url4 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2020/departements/34.csv.gz" # 2020
url5 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2019/departements/34.csv.gz" # 2019

In [70]:
# Load data from URLs
def load_data(url):
    
    df = pd.read_csv(url,compression="gzip", low_memory=False)  
    df.dropna(axis=1, how='all', inplace=True)
    df = df.dropna(subset=['code_postal']) # enlever les valeurs nulles de code postal
    df['code_postal'] = df['code_postal'].astype(int).astype(str)
    #code_mask = df['code_postal'].isin(codes_postaux_metropole) # filtre code postal pour Montpellier Méditerranée Métropole
    #df =df[code_mask]
    df['date_mutation']= pd.to_datetime(df['date_mutation'], format='%Y-%m-%d', errors='coerce')
    df['year'] = df['date_mutation'].dt.year
    return df
    
df0 = load_data(url0)
df1 = load_data(url1)
df2 = load_data(url2)
df3 = load_data(url3)
df4 = load_data(url4)
df5 = load_data(url5)

In [71]:
df0.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,year
0,2024-179483,2024-01-03,1,Vente,500000.0,21.0,NaN,RUE DE LA BERGERIE DE CAUNELLE,0009,34990,...,127.0,4.0,S,sols,NaN,NaN,638.0,3.800856,43.616081,2024
1,2024-179483,2024-01-03,1,Vente,500000.0,21.0,NaN,RUE DE LA BERGERIE DE CAUNELLE,0009,34990,...,NaN,0.0,S,sols,NaN,NaN,638.0,3.800856,43.616081,2024
2,2024-179483,2024-01-03,1,Vente,500000.0,NaN,NaN,LA BERGERIE,B034,34990,...,NaN,NaN,VI,vignes,NaN,NaN,41.0,3.800788,43.616152,2024
3,2024-179484,2024-01-05,1,Vente,40000.0,4.0,NaN,PL DES CHARMILLES,1253,34080,...,17.0,1.0,NaN,NaN,NaN,NaN,NaN,3.837947,43.613467,2024
4,2024-179485,2024-01-03,1,Vente,2170.0,NaN,NaN,RIEU SALAT,B121,34150,...,NaN,NaN,T,terres,NaN,NaN,2712.0,3.565889,43.629017,2024


In [72]:
df0.shape[0], df1.shape[0], df2.shape[0], df3.shape[0], df4.shape[0], df5.shape[0]  

(30085, 77463, 94972, 95505, 73220, 39385)

In [73]:
df_combined = pd.concat([df0, df1, df2, df3, df4, df5], ignore_index=True)


In [74]:
df_combined.shape[0]


410630

In [90]:
select_columns = ['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'numero_volume', 'lot1_surface_carrez',
       'lot2_surface_carrez', 'lot3_surface_carrez', 'lot4_surface_carrez',
       'lot5_surface_carrez', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales']
df_s = df_combined[select_columns]


In [95]:
df12= df_s[df_s['lot2_surface_carrez'].notnull()]  #| df['latitude'].isnull()]
df12

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,numero_volume,lot1_surface_carrez,lot2_surface_carrez,lot3_surface_carrez,lot4_surface_carrez,lot5_surface_carrez,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales
30,2024-179499,2024-01-04,1,Vente,118000.0,0241,34400,34145,Lunel,34,NaN,NaN,51.58,NaN,NaN,NaN,2.0,Appartement,52.0,2.0
31,2024-179499,2024-01-04,1,Vente,118000.0,0241,34400,34145,Lunel,34,NaN,NaN,51.58,NaN,NaN,NaN,3.0,Dépendance,NaN,0.0
64,2024-179517,2024-01-05,1,Vente,105000.0,4690,34090,34172,Montpellier,34,NaN,NaN,27.55,NaN,NaN,NaN,2.0,Appartement,30.0,1.0
65,2024-179517,2024-01-05,1,Vente,105000.0,4690,34090,34172,Montpellier,34,NaN,NaN,27.55,NaN,NaN,NaN,2.0,Appartement,39.0,1.0
201,2024-179585,2024-01-04,1,Vente,140000.0,3665,34070,34172,Montpellier,34,NaN,NaN,65.86,NaN,NaN,NaN,3.0,Dépendance,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408305,2019-278175,2019-11-04,1,Vente,816150.0,0255,34340,34150,Marseillan,34,NaN,42.15,113.10,40.10,NaN,NaN,2.0,Appartement,162.0,3.0
408776,2019-278428,2019-11-25,1,Vente,79000.0,0027,34800,34197,Péret,34,NaN,33.88,12.00,12.00,NaN,NaN,2.0,Appartement,35.0,2.0
408978,2019-278541,2019-12-06,1,Vente,15000.0,0050,34260,34252,Saint-Étienne-Estréchoux,34,NaN,18.21,17.73,NaN,NaN,NaN,2.0,Appartement,36.0,1.0
410350,2019-279201,2019-12-18,1,Vente,84400.0,0375,34120,34199,Pézenas,34,NaN,45.27,45.27,45.27,45.27,NaN,2.0,Appartement,45.0,2.0


In [75]:
df_combined.columns

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'id_parcelle', 'numero_volume',
       'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude',
       'year', 'ancien_code_commune', 'ancien_nom_commune',
       'ancien_id_parcelle'],
      dtype='object')

In [96]:
# Reserve les colonnes suivantes
columns_to_keep = [
    'valeur_fonciere',               # 房價（目標變數）
    'type_local',                    # 房屋類型
    'surface_reelle_bati',           # 建築面積
    'surface_terrain',               # 土地面積
    'nombre_pieces_principales',     # 主要房間數量
    'nombre_lots',                   # 單位數量
    'code_postal',                   # 郵遞區號
    'nom_commune',                   # 城市名稱
    'code_commune',                  # INSEE 城市代碼
    'code_departement',              # 部門代碼
    'longitude',                     # 經度
    'latitude',                      # 緯度
    'date_mutation',                 # 交易日期
    'year',                          # 
]


# filtre les colonnes
df_filtered = df_combined[columns_to_keep]


In [97]:
df_filtered

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,code_departement,longitude,latitude,date_mutation,year,nature_mutation
0,500000.00,Maison,127.0,638.0,4.0,0,34990,Juvignac,34123,34,3.800856,43.616081,2024-01-03,2024,Vente
1,500000.00,Dépendance,NaN,638.0,0.0,0,34990,Juvignac,34123,34,3.800856,43.616081,2024-01-03,2024,Vente
2,500000.00,NaN,NaN,41.0,NaN,0,34990,Juvignac,34123,34,3.800788,43.616152,2024-01-03,2024,Vente
3,40000.00,Appartement,17.0,NaN,1.0,1,34080,Montpellier,34172,34,3.837947,43.613467,2024-01-05,2024,Vente
4,2170.00,NaN,NaN,2712.0,NaN,0,34150,Gignac,34114,34,3.565889,43.629017,2024-01-03,2024,Vente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410625,82333.33,Dépendance,NaN,160.0,0.0,0,34360,Saint-Chinian,34245,34,2.926349,43.398813,2019-10-30,2019,Vente
410626,82333.33,NaN,NaN,146.0,NaN,0,34360,Saint-Chinian,34245,34,2.927151,43.398199,2019-10-30,2019,Vente
410627,35000.00,Appartement,30.0,NaN,2.0,1,34300,Agde,34003,34,3.471147,43.311986,2019-10-17,2019,Vente
410628,208000.00,NaN,NaN,393.0,NaN,0,34370,Maureilhan,34155,34,3.116411,43.353973,2019-09-23,2019,Vente


In [78]:
df_grouped = df_filtered.groupby('type_local').size()
df_grouped

type_local
Appartement                                  83245
Dépendance                                  110082
Local industriel. commercial ou assimilé     13726
Maison                                       63485
dtype: int64

In [98]:
df_type_local = df_filtered[df_filtered['type_local'] .isin(['Appartement','Maison'])]
df_type_local.head()

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,code_departement,longitude,latitude,date_mutation,year,nature_mutation
0,500000.0,Maison,127.0,638.0,4.0,0,34990,Juvignac,34123,34,3.800856,43.616081,2024-01-03,2024,Vente
3,40000.0,Appartement,17.0,NaN,1.0,1,34080,Montpellier,34172,34,3.837947,43.613467,2024-01-05,2024,Vente
8,263660.0,Maison,65.0,NaN,3.0,1,34090,Montpellier,34172,34,3.852466,43.638047,2024-01-05,2024,Vente
11,160500.0,Appartement,43.0,NaN,2.0,2,34090,Montpellier,34172,34,3.868303,43.636036,2024-01-05,2024,Vente
13,250000.0,Appartement,66.0,NaN,3.0,2,34570,Pignan,34202,34,3.770587,43.586034,2024-01-08,2024,Vente


Dépendance : Garage / Cave / Grenier / Abri de Jardin / Piscine / Atelier 
Local industriel, commercial ou assimilé : Local industriel / Local commercial / Asimilé  

In [80]:
df_grouped = df_filtered.groupby('nature_mutation').size()
df_grouped

nature_mutation
Adjudication                            1203
Echange                                 2593
Expropriation                             13
Vente                                 369219
Vente en l'état futur d'achèvement     35971
Vente terrain à bâtir                   1631
dtype: int64

On va garder : Vente /  Vente en l'état futur d'achèvement /Vente terrain à bâtir   

In [99]:
df_vente = df_type_local[df_type_local['nature_mutation'].isin([
    'Vente', 
    'Vente en l\'état futur d\'achèvement', 
    'Vente terrain à bâtir'
])]
df_vente.head()

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,code_departement,longitude,latitude,date_mutation,year,nature_mutation
0,500000.0,Maison,127.0,638.0,4.0,0,34990,Juvignac,34123,34,3.800856,43.616081,2024-01-03,2024,Vente
3,40000.0,Appartement,17.0,NaN,1.0,1,34080,Montpellier,34172,34,3.837947,43.613467,2024-01-05,2024,Vente
8,263660.0,Maison,65.0,NaN,3.0,1,34090,Montpellier,34172,34,3.852466,43.638047,2024-01-05,2024,Vente
11,160500.0,Appartement,43.0,NaN,2.0,2,34090,Montpellier,34172,34,3.868303,43.636036,2024-01-05,2024,Vente
13,250000.0,Appartement,66.0,NaN,3.0,2,34570,Pignan,34202,34,3.770587,43.586034,2024-01-08,2024,Vente


In [100]:
# 1. 檢查每列的缺失值數量
missing_values_per_column = df_vente.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere                 53
type_local                       0
surface_reelle_bati              2
surface_terrain              74652
nombre_pieces_principales        2
nombre_lots                      0
code_postal                      0
nom_commune                      0
code_commune                     0
code_departement                 0
longitude                     1771
latitude                      1771
date_mutation                    0
year                             0
nature_mutation                  0
dtype: int64


In [101]:
# 2. 計算缺失值的總數
total_missing_values = df_vente.isnull().sum().sum()
print(f"Total missing values: {total_missing_values}")

Total missing values: 78251


In [102]:
# 假設 df 是你的 DataFrame
# 查看 'longitude' 和 'latitude' 欄位的缺失值數量
longitude_missing_count = df_vente['longitude'].isnull().sum()
latitude_missing_count = df_vente['latitude'].isnull().sum()

# 計算 'longitude' 和 'latitude' 欄位缺失值的百分比
longitude_missing_percentage = (df_vente['longitude'].isnull().mean()) * 100
latitude_missing_percentage = (df_vente['latitude'].isnull().mean()) * 100

# 顯示結果
print(f"Longitude missing values count: {longitude_missing_count}")
print(f"Longitude missing values percentage: {longitude_missing_percentage}%")
print(f"Latitude missing values count: {latitude_missing_count}")
print(f"Latitude missing values percentage: {latitude_missing_percentage}%")

Longitude missing values count: 1771
Longitude missing values percentage: 1.2135872432862107%
Latitude missing values count: 1771
Latitude missing values percentage: 1.2135872432862107%


In [103]:
df_longitude_latitude = df_vente[df_vente['longitude'].isnull()|df_vente['latitude'].isnull()]

df_longitude_latitude

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,code_departement,longitude,latitude,date_mutation,year,nature_mutation
2097,6500.0,Maison,45.0,NaN,3.0,0,34700,Les Plans,34205,34,NaN,NaN,2024-01-29,2024,Vente
3706,560000.0,Maison,43.0,195.0,2.0,0,34000,Montpellier,34172,34,NaN,NaN,2024-02-20,2024,Vente
5000,8000.0,Maison,178.0,1152.0,6.0,0,34660,Cournonsec,34087,34,NaN,NaN,2024-02-29,2024,Vente
5003,8000.0,Maison,178.0,1200.0,6.0,0,34660,Cournonsec,34087,34,NaN,NaN,2024-02-29,2024,Vente
7477,400000.0,Maison,107.0,1133.0,3.0,0,34660,Cournonsec,34087,34,NaN,NaN,2024-03-14,2024,Vente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409228,180000.0,Appartement,34.0,NaN,2.0,1,34300,Agde,34003,34,NaN,NaN,2019-12-06,2019,Vente
409402,60000.0,Appartement,34.0,NaN,2.0,1,34300,Agde,34003,34,NaN,NaN,2019-11-29,2019,Vente
409710,530000.0,Maison,144.0,878.0,5.0,0,34120,Castelnau-de-Guers,34056,34,NaN,NaN,2019-12-06,2019,Vente
410049,103000.0,Appartement,32.0,NaN,2.0,1,34340,Marseillan,34150,34,NaN,NaN,2019-12-16,2019,Vente


In [108]:
df_vente.dtypes

valeur_fonciere                     float64
type_local                           object
surface_reelle_bati                 float64
surface_terrain                     float64
nombre_pieces_principales           float64
nombre_lots                           int64
code_postal                          object
nom_commune                          object
code_commune                          int64
code_departement                      int64
longitude                           float64
latitude                            float64
date_mutation                datetime64[ns]
year                                  int64
nature_mutation                      object
dtype: object

In [110]:
df_popu = pd.read_csv('herault_population_2019_2024.csv')
df_popu.head()

,code_postal,year,population
0,34091,2024,56
1,34091,2021,54
2,34092,2024,967
3,34092,2023,977
4,34093,2025,147


In [115]:

df_vente['code_postal'] = df_vente['code_postal'].astype(int)

C:\Users\yutin\AppData\Local\Temp\ipykernel_13548\3313786434.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vente['code_postal'] = df_vente['code_postal'].astype(int)


In [116]:
#merge df_popu + de_vente
df_merged = pd.merge(df_vente, df_popu, on=['code_postal', 'year'], how='inner')

In [122]:
df_merged = df_merged.drop(columns=['code_departement', 'nature_mutation'])

In [125]:
df_merged['nombre_pieces_principales'] = df_merged['nombre_pieces_principales'].fillna(0)

In [126]:
df_merged['nombre_pieces_principales'] = df_merged['nombre_pieces_principales'].astype(int)

In [127]:
df_merged

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population
0,40000.0,Appartement,17.0,NaN,1,1,34080,Montpellier,34172,3.837947,43.613467,2024,482
1,135000.0,Appartement,55.0,NaN,2,1,34080,Montpellier,34172,3.826113,43.613047,2024,482
2,88500.0,Appartement,30.0,NaN,1,1,34080,Montpellier,34172,3.840901,43.617317,2024,482
3,140000.0,Appartement,63.0,NaN,3,3,34080,Montpellier,34172,3.849301,43.621547,2024,482
4,82000.0,Appartement,29.0,NaN,2,2,34080,Montpellier,34172,3.841634,43.614330,2024,482
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66868,420000.0,Maison,70.0,2540.0,3,0,34320,Gabian,34109,3.266071,43.500903,2020,2597
66869,420000.0,Maison,70.0,3200.0,3,0,34320,Gabian,34109,3.266071,43.500903,2020,2597
66870,420000.0,Maison,150.0,3200.0,9,0,34320,Gabian,34109,3.266071,43.500903,2020,2597
66871,420000.0,Maison,70.0,1770.0,3,0,34320,Gabian,34109,3.266071,43.500903,2020,2597


In [130]:
df_rev = pd.read_csv('herault_revenu.csv')
df_rev.head()

,code_postal,rev_fisc_moyen_reference
0,34002,21.558788
1,34004,17.478192
2,34006,19.962507
3,34007,17.640876
4,34011,22.605179


In [131]:
#merge df_rev + de_merged
df_merged_rev = pd.merge(df_merged, df_rev, on=['code_postal'], how='inner')

In [132]:
df_merged_rev

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference
0,40000.0,Appartement,17.0,NaN,1,1,34080,Montpellier,34172,3.837947,43.613467,2024,482,21.477236
1,135000.0,Appartement,55.0,NaN,2,1,34080,Montpellier,34172,3.826113,43.613047,2024,482,21.477236
2,88500.0,Appartement,30.0,NaN,1,1,34080,Montpellier,34172,3.840901,43.617317,2024,482,21.477236
3,140000.0,Appartement,63.0,NaN,3,3,34080,Montpellier,34172,3.849301,43.621547,2024,482,21.477236
4,82000.0,Appartement,29.0,NaN,2,2,34080,Montpellier,34172,3.841634,43.614330,2024,482,21.477236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66868,50000.0,Maison,76.0,70.0,3,0,34120,Tourbes,34311,3.378458,43.446364,2020,6756,32.011445
66869,192000.0,Appartement,46.0,NaN,2,1,34120,Pézenas,34199,3.424441,43.459825,2020,6756,32.011445
66870,192000.0,Appartement,46.0,NaN,2,1,34120,Pézenas,34199,3.424441,43.459825,2020,6756,32.011445
66871,192000.0,Maison,180.0,80.0,5,0,34120,Pézenas,34199,3.424535,43.459760,2020,6756,32.011445


In [133]:
df_merged_rev['rev_fisc_moyen_reference'] = df_merged_rev['rev_fisc_moyen_reference'].round(2)

In [137]:
df_merged_rev.shape[0]

66873

In [151]:

missing_values_per_column = df_merged_rev.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere                0
type_local                     0
surface_reelle_bati            0
surface_terrain                0
nombre_pieces_principales      0
nombre_lots                    0
code_postal                    0
nom_commune                    0
code_commune                   0
longitude                    926
latitude                     926
year                           0
population                     0
rev_fisc_moyen_reference       0
dtype: int64


In [139]:
# Delete rows where 'valeur_fonciere' is null
df_merged_rev = df_merged_rev.dropna(subset=['valeur_fonciere'])

In [141]:
df_merged_rev['surface_terrain'] = df_merged_rev['surface_terrain'].fillna(0)

C:\Users\yutin\AppData\Local\Temp\ipykernel_13548\491585317.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_rev['surface_terrain'] = df_merged_rev['surface_terrain'].fillna(0)


In [142]:
df_merged_rev

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference
0,40000.0,Appartement,17.0,0.0,1,1,34080,Montpellier,34172,3.837947,43.613467,2024,482,21.48
1,135000.0,Appartement,55.0,0.0,2,1,34080,Montpellier,34172,3.826113,43.613047,2024,482,21.48
2,88500.0,Appartement,30.0,0.0,1,1,34080,Montpellier,34172,3.840901,43.617317,2024,482,21.48
3,140000.0,Appartement,63.0,0.0,3,3,34080,Montpellier,34172,3.849301,43.621547,2024,482,21.48
4,82000.0,Appartement,29.0,0.0,2,2,34080,Montpellier,34172,3.841634,43.614330,2024,482,21.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66868,50000.0,Maison,76.0,70.0,3,0,34120,Tourbes,34311,3.378458,43.446364,2020,6756,32.01
66869,192000.0,Appartement,46.0,0.0,2,1,34120,Pézenas,34199,3.424441,43.459825,2020,6756,32.01
66870,192000.0,Appartement,46.0,0.0,2,1,34120,Pézenas,34199,3.424441,43.459825,2020,6756,32.01
66871,192000.0,Maison,180.0,80.0,5,0,34120,Pézenas,34199,3.424535,43.459760,2020,6756,32.01


In [144]:
df_merged_rev = df_merged_rev.dropna(subset=['surface_reelle_bati'])

In [ ]:
df_merged_rev['surface_reelle_bati'] = df_merged_rev['surface_reelle_bati'].astype(int)

In [149]:
df_merged_rev['valeur_fonciere'] = df_merged_rev['valeur_fonciere'].astype(int)

C:\Users\yutin\AppData\Local\Temp\ipykernel_13548\2417924950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_rev['valeur_fonciere'] = df_merged_rev['valeur_fonciere'].astype(int)


In [150]:
df_merged_rev

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference
0,40000,Appartement,17,0.0,1,1,34080,Montpellier,34172,3.837947,43.613467,2024,482,21.48
1,135000,Appartement,55,0.0,2,1,34080,Montpellier,34172,3.826113,43.613047,2024,482,21.48
2,88500,Appartement,30,0.0,1,1,34080,Montpellier,34172,3.840901,43.617317,2024,482,21.48
3,140000,Appartement,63,0.0,3,3,34080,Montpellier,34172,3.849301,43.621547,2024,482,21.48
4,82000,Appartement,29,0.0,2,2,34080,Montpellier,34172,3.841634,43.614330,2024,482,21.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66868,50000,Maison,76,70.0,3,0,34120,Tourbes,34311,3.378458,43.446364,2020,6756,32.01
66869,192000,Appartement,46,0.0,2,1,34120,Pézenas,34199,3.424441,43.459825,2020,6756,32.01
66870,192000,Appartement,46,0.0,2,1,34120,Pézenas,34199,3.424441,43.459825,2020,6756,32.01
66871,192000,Maison,180,80.0,5,0,34120,Pézenas,34199,3.424535,43.459760,2020,6756,32.01
